In [6]:
from openpyxl import load_workbook
import pandas as pd
import numpy as np
from openpyxl.utils.dataframe import dataframe_to_rows
from pretty_html_table import build_table
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import time

In [12]:
path = r'C:\Users\Admin\PycharmProjects\pythonProject\SSAU\Project\Input_data.xlsx'  #путь к файлу с данными
wb = load_workbook(path)
ws = wb.worksheets[0]

data = ws.values
columns = next(data)[0:]
df = pd.DataFrame(data, columns=columns)

contacts = {'ABB': None, 'Chint': 'info@chint.ru',     # адреса поставщиков по производителям
            'WAGO': 'info@wago.com', 'КЭАЗ': None}

# получить список обработанных заявок
requests = df['заявка'].value_counts()

# схлопывание одинаковых по кол-ву
df = df.groupby(['наименование'], as_index=False).agg(
    {'производитель': 'first', 'артикул': 'first', 'кол-во': 'sum', 'ед. изм.': 'first'})[
    ['производитель', 'артикул', 'наименование', 'кол-во', 'ед. изм.']].sort_values('производитель')

# разделение на таблички по производителям и отправка табличек адресатам. 
# Данные, для которых нет адреса отправки записываются на след листе в файле .xlsx
wb.create_sheet('result', 1) # создаем новый лист в файле
ws1 = wb['result']        
        
for i in df['производитель'].unique():
    data = (df[df['производитель'] == str(i)].sort_values(['наименование'] ,ascending=[True]))
    table = build_table(data, 'blue_light')
    try:
        send_emails(table, contacts[str(i)])
    except:    
        for r in dataframe_to_rows(data, index=False, header=True):
            ws1.append(r)

# на след листе выведем список номеров отработанных заявок
wb.create_sheet('requests', 2) 
ws2 = wb['requests']            
requests = np.array(requests.index.tolist()).reshape(len(requests), 1)
for r in requests:
    ws2.append(tuple(r))
        
wb.save(path) 


sent
sent


In [2]:
# функция отправки email. На вход принимает тело письма и адрес.
# необходимо внести логин, сервер и пароль.  

def send_emails(tbl, adrs):

        for mail in [adrs,]:
            my_email = 'отправитель@email.com'
            server = smtplib.SMTP('smtp.gmail.com', 587) #587 - порт
            server.starttls()
            time.sleep(1)
            server.login(my_email, "password")  # Получаем доступ

            msg = MIMEMultipart()  # Создаем сообщение
            msg['From'] = my_email  # Отправитель

            msg['Subject'] = 'Запрос счета'  # Тема сообщения
            msg['Return-Receipt-To'] = my_email

            msg['To'] = mail  # Получатель
            body = ("<!DOCTYPE html><head></head><body>Здравствуйте, уважаемый " + mail + 
                    '. Прошу выставить счет на:' + "</body></html>")
            msg.attach(MIMEText(body, 'html'))  # Добавляем в сообщение текст
            msg.attach(MIMEText(tbl, 'html'))
            try:
                server.send_message(msg)  # Отправляем сообщение
            except Exception:
                print("error ", mail)

            else:
                print("sent")

            time.sleep(40) # Задержка, чтобы за спам не забанили

            server.quit()  # Выходим
